In [1]:
import pandas as pd
import numpy as np
import requests
from typing import Any
import os
import sys
from pathlib import Path
from datetime import datetime, timezone
import sqlite3
import pandas as pd
import random
import logging
from evidently import Dataset
from evidently import DataDefinition
from evidently import Report
from evidently.metrics import *
from evidently.presets import *
from evidently.tests import *
import logging
import sqlite3
import pandas as pd
from typing import Tuple
from evidently import MulticlassClassification
import uuid

In [ ]:
def configure_logging():
    """Configure logging handlers and return a logger instance."""
    if Path("logging.conf").exists():
        logging.config.fileConfig("logging.conf")
    else:
        logging.basicConfig(
            format="%(asctime)s [%(levelname)s] %(message)s",
            handlers=[logging.StreamHandler(sys.stdout)],
            level=logging.INFO,
        )

configure_logging()

In [ ]:
def apply_drift() -> pd.DataFrame:
    """
    Apply a random drift to the body mass measurements in the penguins dataset.

    This function reads the `data/penguins.csv` file into a pandas DataFrame,
    removes the 'species' column, and introduces a stochastic drift to the
    'body_mass_g' column. The drift is uniformly distributed between 1 and
    three times the standard deviation of the original 'body_mass_g' values.
    The modified DataFrame is then returned.

    Returns
    -------
    pd.DataFrame
        A DataFrame containing the penguins dataset with the 'species' column
        removed and the 'body_mass_g' values perturbed by a random drift.

    Notes
    -----
    - The function assumes that `data/penguins.csv` exists and contains a
      'body_mass_g' column.
    - The random number generator used is `numpy.random.default_rng()`, ensuring
      reproducibility control if desired.
    """

    # load the original dataset from the data direcory in the projects root
    data = pd.read_csv("data/penguins.csv")
    # remove the ground truth label
    data.pop('species')
    # instantiate a uniformly distributed randon number generate
    rng = np.random.default_rng()
    # add a random number between 1 and 3 times its own std to body_mass_g,
    # flipper_length_mm, bill_depth_mm and bill_length_mm
    data['body_mass_g'] += rng.uniform(
        1,
        3 * data["body_mass_g"].std(),
        size=len(data)
    )

    data["flipper_length_mm"] += rng.uniform(
        1,
        3 * data["flipper_length_mm"].std(),
        size=len(data)
    )

    data["bill_depth_mm"] += rng.uniform(
        1,
        3 * data["bill_depth_mm"].std(),
    )
    
    data["bill_length_mm"] += rng.uniform(
        1,
        3 * data["bill_length_mm"].std(),
    )

    return data



In [ ]:
drift_features = apply_drift()

In [ ]:
def generate_classifications(payload: dict[str, Any]) \
        -> dict[str, Any] | list[Any] | None:
    """
    Send a classification request to the inference endpoint and return the results.

    This function makes a POST request to a local inference endpoint with the
    provided payload and returns the classification results if successful.

    Parameters
    ----------
    payload : dict[str, Any]
        The input data to be classified. Should contain an 'inputs' key with
        the data formatted according to the endpoint's expected schema.

    Returns
    -------
    dict[str, Any] or list[Any] or None
        The classification results returned by the inference endpoint if the
        request is successful (status code 200). Returns None if the request
        fails or encounters an error.

    Notes
    -----
    - The function sends requests to http://0.0.0.0:8080/invocations
    - Uses JSON content type for the request
    - Logs error information if the response status code is not 200
    - Does not raise exceptions; returns None on failure

    Examples
    --------
    payload = {"inputs": [{"feature1": 1.0, "feature2": 2.0}]}
    result = generate_classifications(payload)
    print(result)
    {'predictions': [0.85, 0.15]}

    invalid_payload = {}
    result = generate_classifications(invalid_payload)
    Error: 400, text: Invalid input format
    print(result)
    None

    See Also
    --------
    generate_traffic : Uses this function to generate classifications in batches
    """
    # locally hosted model URL
    url = "http://0.0.0.0:8080/invocations"
    headers = {"Content-Type": "application/json"}
    response = requests.post(url, json=payload, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        logging.info("Error: %s, text: s%", response.status_code, response.text)
        return None


In [ ]:
def capture_traffic(model_input: pd.DataFrame,
                    classifications: dict, data_uri: str) -> None:
    """
    Store model input data and predictions in a SQLite database.

    This function captures inference traffic by saving the input features,
    predictions, and metadata (timestamps and UUIDs) to a SQLite database
    for monitoring and analysis purposes.

    Parameters
    ----------
    model_input : pd.DataFrame
    The input features used for model inference. This DataFrame is copied
    and augmented with additional columns before storage.
    classifications : list
        The classification results from the model. Expected to be a dictionary-like
        object with a 'predictions' key containing a list of prediction values.
        If None or empty, the classification column will contain None values.
    data_uri : str
        The file path or URI to the SQLite database where the data will be stored.
        The data is appended to a table named 'data'.

    Returns
    -------
    None
        This function does not return a value. Data is stored directly in the
        specified database.

    Notes
    -----
    - Creates a copy of the input DataFrame to avoid modifying the original
    - Adds the following columns to the data before storage:
        * date: Current UTC timestamp
        * classification: Model predictions from the classifications parameter
        * ground_truth: Initialized as None (for later labeling)
        * uuid: Unique identifier for each row (UUID4 format)
    - Appends data to the 'data' table in the SQLite database
    - Logs errors if database operations fail but does not raise exceptions
    - Ensures the database connection is properly closed in all cases

    Examples
    --------
    import pandas as pd
    model_input = pd.DataFrame({'feature1': [1.0, 2.0], 'feature2': [3.0, 4.0]})
    classifications = {'predictions': [0, 1]}
    capture_traffic(model_input, classifications, 'inference_data.db')
    # Stores the input and predictions in inference_data.db

    # Handle case with no predictions
    capture_traffic(model_input, None, 'inference_data.db')
    # Stores input with None values for classifications

    See Also
    --------
    generate_traffic : Uses this function to store batches of inference data
    """
    logging.info("Storing input payload and predictions in the database...")
    connection = None
    try:
        # connect to the database
        connection = sqlite3.connect(data_uri)
        # assign input data to variable 'data'
        data = model_input.copy()
        # add date column to DatFrame
        data["date"] = datetime.now(timezone.utc)
        #add classification column to DataFrame
        data["classification"] = None
        # add ground_truth column to DataFrame
        data["ground_truth"] = None
        # add model classification to drift DataFrame
        if classifications is not None and len(classifications) > 0:
            data["classification"] = [item['classification'] for item in classifications['predictions']]

        # add unique id to the DataFrame
        data["uuid"] = [str(uuid.uuid4()) for _ in range(len(data))]
        # save DataFrame to database
        data.to_sql("data", connection, if_exists="append", index=False)

    except sqlite3.Error:
        logging.exception(
            "There was an error saving the input request and output prediction "
            "in the database.",
        )
    finally:
        if connection:
            connection.close()

In [ ]:
def generate_traffic(data: pd.DataFrame, samples: int) -> None:
    """
    Generate synthetic traffic by sampling data and creating classifications.

    This function repeatedly samples batches of data, generates classifications
    for each batch, and stores the results in a SQLite database. It continues
    until the specified number of samples have been processed.

    Parameters
    ----------
    data : pd.DataFrame
        The source DataFrame from which to sample data for classification.
        Must contain the required features for generating classifications.
    samples : int
        The target number of samples to generate. The function will continue
        processing batches until at least this many classifications have been
        created.

    Returns
    -------
    None
        This function does not return a value. Results are stored in the
        'inference_data.db' SQLite database.

    Notes
    -----
    - Samples data in batches of 10 rows
    - Replaces NaN, inf, and -inf values with None before processing
    - Stores classification results in 'inference_data.db' via the
        capture_traffic function
    - Logs progress and errors throughout execution
    - Queries and logs the last 5 database entries upon completion

    Examples
    --------
    import pandas as pd
    df = pd.DataFrame({'feature1': [1, 2, 3], 'feature2': [4, 5, 6]})
    generate_traffic(df, samples=100)
    # Generates 100 classifications and stores them in the database

    See Also
    --------
    generate_classifications : Generates classifications from input payload
    capture_traffic : Stores batch data and classifications in the database
    """

    try:
        classifications_generated = 0
        # send data to the endpoint while classifications are less than classifications
        # generated by the model
        while classifications_generated < samples:
            # instantiate dictionary called payload
            payload = {}
            #sample from the drift dataset replacing NaN values with none to ensure
            #compatibility with JSON objects
            batch = data.sample(n=10).replace([np.nan, np.inf, -np.inf], None)
            # generate a dictionary of inputs to send to the enpoint
            payload["inputs"] = [
                {
                    k: (None if pd.isna(v) else v)
                    for k, v in row.to_dict().items()
                }
                for _, row in batch.iterrows()
            ]
            # send inputs to the endpoint
            classifications = generate_classifications(payload)
            logging.info(f"Generated classifications %s", classifications)
            try:
                # capture the input data and classifications and save it to the database
                capture_traffic(batch, classifications,"inference_data.db")
            except sqlite3.Error:
                logging.exception(
                    "There was an error connecting to the database. ",
                )
            # update the count of classifications generated
            classifications_generated += len(batch)
            logging.info("Generated %s classifications",classifications_generated)

    except Exception:
        logging.exception("There was an error sending traffic to the endpoint.")

In [ ]:
generate_traffic(drift_features, 400)

# Labelling

In [ ]:
def retrieve_data() -> pd.DataFrame:
    """
    Retrieve all unlabeled data from the inference database.

    Connects to the SQLite database, executes a query to fetch all records
    from the data table where ground_truth is NULL (unlabeled data), and
    returns the results as a pandas DataFrame. Records are ordered by date
    in descending order (newest first).

    Parameters
    ----------
    None

    Returns
    -------
    pd.DataFrame
        DataFrame containing all unlabeled records with the following columns:
        - uuid : unique identifier for each record
        - island : island location
        - sex : sex of the penguin
        - bill_length_mm : bill length in millimeters
        - bill_depth_mm : bill depth in millimeters
        - flipper_length_mm : flipper length in millimeters
        - body_mass_g : body mass in grams
        - classification : model prediction
        - ground_truth : actual classification label (NULL for unlabeled data)

        Records are sorted by date in descending order.

    Notes
    -----
    The function establishes a connection to 'inference_data.db', retrieves
    only data where ground_truth is NULL (hasn't been labeled yet), and
    properly closes the connection before returning.

    Examples
    --------
    data = retrieve_data()
    print(data.head())
    print(data['ground_truth'].isna().all())  # Should return True
    """
    # Establish connection to the SQLite database
    connection = sqlite3.connect("inference_data.db")

    logging.info("Retrieving data....")

    # SQL query to select all relevant columns from the data table
    # Only retrieve records where ground_truth is NULL (unlabeled)
    # Results are ordered by date in descending order (newest first)
    query = (
        "SELECT uuid, island, sex, bill_length_mm, bill_depth_mm, flipper_length_mm, "
        "body_mass_g, classification, ground_truth FROM data "
        "WHERE ground_truth IS NULL "
        "ORDER BY date DESC;"
    )

    # Execute query and load results into a pandas DataFrame
    data = pd.read_sql_query(query, connection)

    logging.info("Retrieved data")

    # Close the database connection
    connection.close()

    return data

In [ ]:
def get_label(prediction: str, prediction_quality: float) -> str:
    """
    Generate a simulated ground truth label based on prediction quality.

    This function creates synthetic ground truth labels for testing and simulation
    purposes. It returns the predicted label with a probability equal to the
    prediction quality, otherwise returns a random label from the possible classes.

    Parameters
    ----------
    prediction : str
        The model's predicted label. Should be one of the valid penguin species:
        "Adelie", "Chinstrap", or "Gentoo".
    prediction_quality : float
        The desired accuracy rate for the predictions, expressed as a probability
        between 0.0 and 1.0. A value of 0.8 means the ground truth will match
        the prediction 80% of the time.

    Returns
    -------
    str
        A ground truth label. Returns the input prediction with probability equal
        to `prediction_quality`, otherwise returns a randomly selected label from
        ["Adelie", "Chinstrap", "Gentoo"].

    Notes
    -----
    - This function is intended for simulation and testing purposes only
    - Uses random selection, so results are non-deterministic
    - Does not validate that the prediction is a valid penguin species
    - The random label may coincidentally match the prediction even when
      the quality check fails

    Examples
    --------
    # With high prediction quality, usually returns the prediction
    label = get_label("Adelie", prediction_quality=0.9)
    # Returns "Adelie" 90% of the time, random species 10% of the time

    # With low prediction quality, often returns a different label
    label = get_label("Chinstrap", prediction_quality=0.3)
    # Returns "Chinstrap" 30% of the time, random species 70% of the time

    # Simulate perfect predictions
    label = get_label("Gentoo", prediction_quality=1.0)
    print(label)
    'Gentoo'

    See Also
    --------
    random.choice : Used to select random labels when prediction quality check fails
    """

    return (
        prediction
        if random.random() < prediction_quality
        else random.choice(["Adelie", "Chinstrap", "Gentoo"])
        )


In [ ]:
def label_data(data: pd.DataFrame) -> int|None:
    """
    Generate and store simulated ground truth labels for inference data.

    This function creates synthetic ground truth labels for the provided data
    and updates the corresponding records in the database. Labels are generated
    with a simulated accuracy of 80% using the get_label function.

    Parameters
    ----------
    data : pd.DataFrame
        A DataFrame containing inference records to label. Must include 'uuid'
        and 'classification' columns. The 'uuid' column is used to identify
        records in the database for updating.

    Returns
    -------
    None or int
        Returns 0 if the input DataFrame is empty, otherwise returns None after
        successfully updating the database with ground truth labels.

    Notes
    -----
    - Connects to 'inference_data.db' SQLite database
    - Updates the 'ground_truth' column in the 'data' table
    - Uses a fixed prediction quality of 0.8 (80% accuracy) for label generation
    - Commits all updates in a single transaction after processing all rows
    - Logs the labeling process for monitoring purposes
    - Automatically closes the database connection after updates

    Examples
    --------
    import pandas as pd
    data = pd.DataFrame({
    ...     'uuid': ['123e4567-e89b-12d3-a456-426614174000'],
    ...     'classification': ['Adelie']
    ... })
    label_data(data)
    # Updates the database with ground truth labels

    # Handle empty DataFrame
    empty_data = pd.DataFrame()
    result = label_data(empty_data)
    print(result)
    0

    See Also
    --------
    get_label : Generates individual ground truth labels based on predictions
    retrieve_data : Retrieves inference data from the database for labeling
    """

    # Establish connection to the SQLite database
    connection = sqlite3.connect("inference_data.db")

    # Check if the input DataFrame is empty and return early if so
    if data.empty:
        return 0

    logging.info("Generating ground truth labels...")

    # Iterate through each row in the DataFrame to generate and store labels
    for _, row in data.iterrows():
        # Extract the unique identifier for the current record
        uuid = row["uuid"]

        # Generate a simulated ground truth label with 80% accuracy
        # based on the model's classification
        label = get_label(row["classification"], 0.8)

        # Prepare SQL query to update the ground_truth column for this record
        update_query = "UPDATE data SET ground_truth = ? WHERE uuid = ?"

        # Execute the update query with the generated label and uuid
        connection.execute(update_query, (label, uuid))

    # Commit all updates to the database in a single transaction
    connection.commit()

    # Close the database connection to free resources
    connection.close()

    logging.info("Generated ground truth labels")

    return None


In [ ]:
drift_inference_data = retrieve_data()

In [ ]:
label_data(drift_inference_data)

#Monitoring

In [ ]:
def retrieve_labelled_data(samples: int) -> pd.DataFrame:
    connection = sqlite3.connect("inference_data.db")
    logging.info("Retrieving data....")

    query = (
                "SELECT island, sex, bill_length_mm, bill_depth_mm, flipper_length_mm, "
                "body_mass_g, classification, ground_truth FROM data "
                "ORDER BY date DESC LIMIT ?;"
            )
    data = pd.read_sql_query(query, connection, params=(samples,))
    logging.info("Retrieved data")
    connection.close()

    return data

In [ ]:
def create_datasets() -> Tuple[Dataset, Dataset]:
    """
    Create reference and current Evidently AI datasets for model monitoring.

    This function prepares two datasets for monitoring: a reference dataset from
    historical penguin data and a current dataset from recent inference results.
    Both datasets are configured with the same schema defining numerical and
    categorical features.

    Parameters
    ----------
    None

    Returns
    -------
    evidently_training_dataset : Dataset
        The reference Evidently AI dataset created from the historical penguin
        data (penguins.csv). The 'species' column is renamed to 'ground_truth'
        and duplicated as 'classification' to match the monitoring schema.
    evidently_inference_dataset : Dataset
        The current Evidently AI dataset created from the 100 most recent
        inference records retrieved from the database.

    Notes
    -----
    - Defines a schema with the following structure:
        * Numerical columns: bill_length_mm, bill_depth_mm, flipper_length_mm,
          body_mass_g
        * Categorical columns: classification, ground_truth, island, sex
    - Reference data is loaded from 'data/penguins.csv'
    - Current data retrieves the 100 most recent records from the database
    - Reference dataset has 'species' renamed to 'ground_truth' and copied to
      'classification' for consistency with inference data format
    - Both datasets use the same data definition schema for comparability
    - Logs the dataset creation process

    Examples
    --------
    ref_dataset, curr_dataset = create_datasets()
    print(ref_dataset.data.columns.tolist())
    ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g',
     'classification', 'ground_truth', 'island', 'sex']

    See Also
    --------
    retrieve_data : Retrieves current inference data from the database
    Dataset.from_pandas : Creates Evidently AI datasets from pandas DataFrames
    DataDefinition : Defines the schema for Evidently AI datasets
    """
    logging.info("Creating datasets")
    # Create a monitoring schema for the dataset
    schema = DataDefinition(
        numerical_columns=[
            "bill_length_mm",
            "bill_depth_mm",
            "flipper_length_mm",
            "body_mass_g",
        ],
        categorical_columns=[
            "classification",
            "ground_truth",
            "island",
            "sex",
        ],
        classification=[MulticlassClassification(
            target="ground_truth",
            prediction_labels="classification",
        )]
    )
    # create a DataFrame with historical data
    training_data = pd.read_csv("data/penguins.csv")
    # rename the species column as ground_truth for consistency with inference
    # data
    training_data.rename(columns={"species":"ground_truth"}, inplace=True)
    # create a classification column to match monitoring schema
    training_data["classification"] = training_data["ground_truth"]
    # retrieve inference data from database
    inference_data = retrieve_labelled_data(200)
    # create dataset object with the historical data
    evidently_training_dataset = Dataset.from_pandas(
        training_data,
        data_definition=schema
    )
    # Create a dataset object with the inference data
    evidently_inference_dataset = Dataset.from_pandas(
        inference_data,
        data_definition=schema
    )

    # return a tuple of the historical and inference dataset objects
    return evidently_training_dataset, evidently_inference_dataset

In [ ]:
evidently_training_dataset, evidently_inference_dataset = create_datasets()

In [ ]:
def column_report(training_dataset: Dataset, inference_dataset: Dataset) -> Report:
    """
    Run column-level statistical reports on selected features.

    This function computes column-specific metrics comparing inference data
    against a training (reference) dataset using Evidently AI metric primitives.
    The report focuses on median values for numerical columns and unique value
    counts for categorical columns.

    Parameters
    ----------
    training_dataset : Dataset
        The reference Evidently AI dataset, typically derived from training
        or historical data, used as the baseline for comparison.
    inference_dataset : Dataset
        The current Evidently AI dataset containing recent inference data
        to be evaluated against the reference dataset.

    Returns
    -------
    EvaluationResult
        The result of running the Evidently AI report, containing computed
        column-level statistics and comparison metrics.

    Notes
    -----
    - Computes median values for numerical columns:
        * bill_length_mm
        * bill_depth_mm
        * flipper_length_mm
        * body_mass_g
    - Computes unique value counts for categorical columns:
        * island
        * sex
    - Compares inference data against training data
    - Returns the executed report results to the caller

    Examples
    --------
    train_data, infer_data = create_datasets()
    eval_result = column_report(train_data, infer_data)
    # Inspect column-level metrics in eval_result

    See Also
    --------
    Report : Evidently AI report class for generating analyses
    MedianValue : Metric for computing median values of numeric columns
    UniqueValueCount : Metric for counting unique values in a column
    """

    
    column_report = Report([
        MedianValue(column="bill_length_mm"),
        MedianValue(column="bill_depth_mm"),
        MedianValue(column="flipper_length_mm"),
        MedianValue(column="body_mass_g"),
        UniqueValueCount(column="island"),
        UniqueValueCount(column="sex"),
    ])
    
    # Execute the evaluations
    column_report = column_report.run(inference_dataset, training_dataset)
    
    return column_report
    

In [ ]:
column_report = column_report(evidently_training_dataset, evidently_inference_dataset)

column_report

In [ ]:
def dataset_report(training_dataset: Dataset, inference_dataset: Dataset) -> Report:
    """
    Run dataset-level summary statistics reports.

    This function computes high-level dataset metrics for both training
    (reference) and inference datasets using Evidently AI dataset-level
    metrics. The report captures basic structural and completeness
    characteristics of the datasets.

    Parameters
    ----------
    training_dataset : Dataset
        The reference Evidently AI dataset, typically derived from training
        or historical data.
    inference_dataset : Dataset
        The current Evidently AI dataset containing recent inference data
        to be evaluated.

    Returns
    -------
    EvaluationResult
        The result of running the Evidently AI report, containing computed
        dataset-level statistics and metrics.

    Notes
    -----
    - Computes the following dataset-level metrics:
        * Row count
        * Column count
        * Number of empty columns
        * Number of empty rows
    - Evaluates metrics for both training and inference datasets
    - Returns the executed report results to the caller

    Examples
    --------
    train_data, infer_data = create_datasets()
    eval_result = dataset_report(train_data, infer_data)
    # Inspect dataset-level metrics in eval_result

    See Also
    --------
    Report : Evidently AI report class for generating analyses
    RowCount : Metric for counting dataset rows
    ColumnCount : Metric for counting dataset columns
    EmptyColumnsCount : Metric for counting empty columns
    EmptyRowsCount : Metric for counting empty rows
    """

    dataset_report = Report([
        RowCount(),
        ColumnCount(),
        EmptyColumnsCount(),
        EmptyRowsCount(),  
    ])

    dataset_report = dataset_report.run(training_dataset, inference_dataset)
    
    return dataset_report

In [ ]:
dataset_report = dataset_report(evidently_training_dataset, evidently_inference_dataset)

dataset_report

In [ ]:
def column_report_with_tests(training_dataset: Dataset, inference_dataset: Dataset) -> Report:
    """
    Run column-level statistical reports with automated tests enabled.

    This function computes column-specific metrics comparing inference data
    against a training (reference) dataset using Evidently AI metric primitives,
    and includes built-in statistical tests for detecting significant changes
    between datasets.

    Parameters
    ----------
    training_dataset : Dataset
        The reference Evidently AI dataset, typically derived from training
        or historical data, used as the baseline for comparison.
    inference_dataset : Dataset
        The current Evidently AI dataset containing recent inference data
        to be evaluated against the reference dataset.

    Returns
    -------
    EvaluationResult
        The result of running the Evidently AI report, containing computed
        column-level statistics and associated test results.

    Notes
    -----
    - Computes median values for numerical columns:
        * bill_length_mm
        * bill_depth_mm
        * flipper_length_mm
        * body_mass_g
    - Computes unique value counts for categorical columns:
        * island
        * sex
    - Enables Evidently AI automated tests via `include_tests=True`
    - Compares inference data against training data
    - Returns the executed report results to the caller

    Examples
    --------
    train_data, infer_data = create_datasets()
    eval_result = column_report_with_tests(train_data, infer_data)
    # Inspect metrics and test outcomes in eval_result

    See Also
    --------
    Report : Evidently AI report class for generating analyses
    MedianValue : Metric for computing median values of numeric columns
    UniqueValueCount : Metric for counting unique values in a column
    """

    
    column_report = Report([
        MedianValue(column="bill_length_mm"),
        MedianValue(column="bill_depth_mm"),
        MedianValue(column="flipper_length_mm"),
        MedianValue(column="body_mass_g"),
        UniqueValueCount(column="island"),
        UniqueValueCount(column="sex"),
    ],
    include_tests=True)
    
    # Execute the evaluations
    column_report = column_report.run(inference_dataset, training_dataset)
    
    return column_report

In [ ]:
column_tests = column_report_with_tests(evidently_training_dataset, evidently_inference_dataset)

column_tests

In [ ]:
def run_drift_report(ev_ref_data: Dataset, ev_curr_data: Dataset) -> Report:
    """
    Run an Evidently AI data drift and classification report.

    This function compares current inference data against reference data using
    Evidently AI presets for data drift and classification performance. The
    report is executed and the evaluation results are returned to the caller.

    Parameters
    ----------
    ev_ref_data : Dataset
        The reference Evidently AI dataset used as the baseline for drift
        detection. Typically contains historical or training data.
    ev_curr_data : Dataset
        The current Evidently AI dataset containing recent inference data
        to be compared against the reference data.

    Returns
    -------
    EvaluationResult
        The result of running the Evidently AI report, containing computed
        drift metrics and classification statistics.

    Notes
    -----
    - Uses `DataDriftPreset` with a threshold of 0.1 (10%)
    - Includes `ClassificationPreset` for classification-related metrics
    - Compares current data against reference data to detect distribution shifts
    - Logs the start of the report execution process

    Examples
    --------
    ref_data, curr_data = create_datasets()
    eval_result = run_report(ref_data, curr_data)
    # Access drift metrics from eval_result

    See Also
    --------
    Report : Evidently AI report class for generating analyses
    DataDriftPreset : Preset configuration for data drift detection
    ClassificationPreset : Preset configuration for classification metrics
    """

    logging.info("Running report...")
    # Create a report with pre-built evaluation templates for drift
    # and classifications tasks
    drift_report = Report([
    DataDriftPreset(
    threshold=0.1,
        ),
    ClassificationPreset()]
    )
    # Execute the evaluations
    my_eval = drift_report.run(ev_curr_data, ev_ref_data)
    #Add the evaluation to the current project

    return my_eval

In [ ]:
drift_report = run_drift_report(evidently_training_dataset, evidently_inference_dataset)

drift_report

In [ ]:
drift_features = apply_drift()

In [ ]:
generate_traffic(drift_features, 400)

In [ ]:
drift_inference_data = retrieve_data()

In [ ]:
label_data(drift_inference_data)

In [ ]:
evidently_training_dataset, evidently_inference_dataset = create_datasets()

In [ ]:
column_report = column_report(evidently_training_dataset, evidently_inference_dataset)

column_report

In [ ]:
my_eval = run_report(evidently_training_dataset, evidently_inference_dataset, ws, project)

my_eval

In [ ]:
my_eval = run_report_test(evidently_training_dataset, evidently_inference_dataset)

my_eval

In [ ]:
column_tests = column_report_with_tests(evidently_training_dataset, evidently_inference_dataset)

column_tests